# 대화형 챗봇이란?

인공지능을 생각할 때 인간의 언어를 이해하고 서로 소통할 수 있는 기계를 자연스럽게 떠올리곤 한다. 그리고 우리 주변에서 볼 수 있는 챗봇은 모두 대화형이 아니다. [챗봇의 5가지 유형](https://tonyaround.com/%ec%b1%97%eb%b4%87-%ea%b8%b0%ed%9a%8d-%eb%8b%a8%ea%b3%84-%ec%b1%97%eb%b4%87%ec%9d%98-5%ea%b0%80%ec%a7%80-%eb%8c%80%ed%91%9c-%ec%9c%a0%ed%98%95-%ec%a2%85%eb%a5%98/)을 보면 대화형, 트리형, 추천형, 시나리오형, 이들을 결합한 결합형 챗봇이 있다. 대화형 챗봇이 아니라면 한계는 명확하게 있다. 진정한 팻봇이면 사용자가 어떤 말을 하더라도 알아듣고 적절하게 대응할 수 있는 자유도가 있어야 하기 때문이다. 그래서 딥러닝을 통한 자연어처리 기술이 가능성을 보여주자, 사람들은 대화형 챗봇이 가져올 변화에 기대감을 가지기 시작했다.   

__챗봇과 딥러닝__   
하지만 우리 주변에 있는 챗봇은 스마트하지 않다. 왜일까? [챗봇 역사의 모든 것](https://blog.performars.com/ko/%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5-%EC%B1%97%EB%B4%87chatbot-%EC%B1%97%EB%B4%87-%EC%97%AD%EC%82%AC%EC%9D%98-%EB%AA%A8%EB%93%A0-%EA%B2%83)을 확인해보면 챗봇의 발전 흐름을 볼 수 있다.   
초창기 챗봇에 대한 기대, 한계점을 느낀 후 최근 ALBERT, BERT, ULMFiT, Transformer-XL 등 tansformer 모델을 활용해 pretrain을 적용한 NLP 모델들이 각광을 받고 있다.   

트랜스포머 모델을 기반으로 인코더-디코더 구조를 바탕으로 챗봇을 제작해보자. 물론 좋은 성능을 내기 위해 수많은 코퍼스로 pretrained model을 활용하는게 필요하지만 오늘은 모델의 기본 구조를 알아보는데 초점을 맞추고 알아보자.   

```
$ mkdir -p ~/aiffel/songys_chatbot
```

# 12-2. 트랜스포머와 인코더 디코더

![](https://d3s0tskafalll9.cloudfront.net/media/images/Untitled_UcFQAjh.max-800x600.png)

번역기는 인코더와 디코더 두 가지 아키텍처로 구성되어있다. 인코더에 입력문장, 디코더에 출력문장. 이를 훈련하는것은 입력, 출력 두 가지 병렬 구조로 구성된 데이터셋을 훈련한다는 의미다. 이러한 구조는 번역기가 아닌 주어진 질문에 답변할 수 있는 챗봇으로 만들 수 있다.

트랜스포머도 입력 문장을 넣으면 출력 문장을 내뱉는 인코더와 디코더 구성을 갖고 있다.   
![](https://d3s0tskafalll9.cloudfront.net/media/images/Untitled_2_EnQyi4S.max-800x600.png)

초록색이 encoder layer, 핑크색이 decoder layer인데 입력 문장은 누적된 인코더의 층을 통해 정보를 뽑아내고 디코더는 누적된 디코더 층을 통해 출력 문장의 단어를 하나씩 만들어 가는 구조를 갖고 있다. 이 과정을 더 확대하면 다음과 같다.   
![](https://d3s0tskafalll9.cloudfront.net/media/images/Untitled_3_ddZedfW.max-800x600.png)

# 12-3. 트랜스포머의 입력 이해하기

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import re
import numpy as np
import matplotlib.pyplot as plt
print("슝=3")

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_4_fuzN6PD.png)

많은 자연어 처리 모델들은 텍스트 문장을 입력으로 받기 위해 단어를 임베딩 벡터로 변환하는 벡터화 과정을 거친다. 트랜스포머 또한 그 점에서는 다른 모델들과 다르지 않다. 하지만 트랜스포머 모델의 입력 데이터 처리는 RNN 계열의 모델들과 달리 임베딩 벡터에 어떤 값을 더해준 뒤 입력으로 사용한다.위 그림의 Positional Encoding에 해당하는 부분이다.   
![](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_5_kH52kQN.png)

이렇게 하는 이유는 입력 받을 때 문장에 있는 단어들을 1개씩 순차적으로 받는게 아니라 모든 단어를 한꺼번에 입력으로 받기 때문이다. 트랜스포머와 RNN의 결정적인 차이점이 이 부분이다. RNN은 문장을 구성하는 단어들이 어순대로 모델에 입력되어 어순 정보를 알려줄 필요가 없었는데, 같은 단어라도 어순에 따라 의도하는 정보가 달라질 수 있음을 유의하여 위치를 알려주는 벡터값을 더해 모델의 입력으로 삼는다.   
![](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_6_DyxB6Ax.png)   
$d_{model}$은 임베딩 벡터의 차원을 의미하고, $pos$는 입력 문장에서 입베딩 벡터의 위치를 나타내며, $i$는 임베딩 벡터 내의 차원의 인덱스를 의미한다. 임베딩 행렬과 포지셔널 행렬이라는 두 행렬을 더함으로써 각 단어 벡터에 위치 정보를 더해주게 된다.   
포지셔널 행렬을 확인해보자.

In [ ]:
# 포지셔널 인코딩 레이어
class PositionalEncoding(tf.keras.layers.Layer):

  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    # 각도 배열 생성
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)

    # 배열의 짝수 인덱스에는 sin 함수 적용
    sines = tf.math.sin(angle_rads[:, 0::2])
    # 배열의 홀수 인덱스에는 cosine 함수 적용
    cosines = tf.math.cos(angle_rads[:, 1::2])

    # sin과 cosine이 교차되도록 재배열
    pos_encoding = tf.stack([sines, cosines], axis=0)
    pos_encoding = tf.transpose(pos_encoding,[1, 2, 0]) 
    pos_encoding = tf.reshape(pos_encoding, [position, d_model])

    pos_encoding = pos_encoding[tf.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

print("슝=3")

In [ ]:
sample_pos_encoding = PositionalEncoding(50, 512)

plt.pcolormesh(sample_pos_encoding.pos_encoding.numpy()[0], cmap='RdBu')
plt.xlabel('Depth')
plt.xlim((0, 512))
plt.ylabel('Position')
plt.colorbar()
plt.show()

행의 크기가 50, 열의 크기가 512인 행렬을 그렸다. 이는 최대 문장의 길이가 50, 임베딩 차원을 512로 하는 모델의 입력 벡터 모양과 같다. 실제 논문에서는 다음과 같이 그림으로 표현하고 있다.   
![](https://d3s0tskafalll9.cloudfront.net/media/images/Untitled_9_l58gVWT.max-800x600.png)

# 12-4. 어텐션

어텐션 메커니즘을 그림으로 표현하면 다음과 같다.   
![](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_10_AaCfqrY.png)   
주어진 query에 대해 모든 key와의 유사도를 각각 구한다. 그리고 구한 유사도를 key와 맵핑되어있는 각각의 값에 반여애준다. 그리고 유사도가 반영된 값을 모두 더해서 뭉쳐주면 어텐션 값이라고 한다.

트랜스포머는 세 가지의 어텐션을 사용한다.   
![](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_11_tFFhFjx.png)   
- 인코더 셀프 어텐션은 인코더에서 인코더의 입력으로 들어간 문장 내 단어들이 서로 유사도를 구한다.
- 디코더 셀프 어텐션은 디코더에서 단어 1개씩 생성하는 디코더가 이미 생성된 앞 단어들과 유사도를 구한다.
- 인코더-디코더 어텐션은 디코더에서 잘 예측하기 위해 인코더에 입력된 단어들과 유사도를 구한다.

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_12_SIe2V15.png)

트랜스포머 어텐션 함수에 사용되는 query, key, value는 기본적으로 정보를 함축한 단어 벡터이다. 여기서 단어 벡터란 초기 입력으로 사용된 임베딩 벡터가 아닌, 트랜스포머의 여러 연산을 거친 단어 벡터이다. 어텐션중 두개가 셀프 어텐션인데 이의 의미를 알아보자.

셀프 어텐션은 유사도를 구하는 대상이 다른 문장의 단어가 아닌 현재 문장 내의 단어들을 말한다. 위에서 말한 인코더-디코더 어텐션은 서로 다른 단어 목록 사이에서 유사도를 구한다.   
우리는 이것, 저것에 대한 의미를 문장에 있는 단어들로 통해 쉽게 유추한다. 하지만 기계는 그렇지 않다. 그러므로 셀프 어텐션을 통해 유사도를 구하고 이를 통해 무엇과 연관되어 있는지 확률을 구할 수 있다.

# scaled dot product attention

어텐션이 단어들간 유사도를 구하는 메커니즘인데 유사도를 어떻게 구할까?   
$Attention(Q,K,V) = softmax(\frac{QK^T}{\sqrt{d_k}})V$   
$Q,K,V$는 순서대로 query, key, value를 말한다. 어텐션 함수는 query에 대한 모든 key의 유사도를 각각 구한다. 그리고 구한 유사도를 키와 맵핑된 각각의 값에 반영한다. 그리고 유사도가 반영된 값을 모두 더해주면 최종적으로 어텐션 값이 나온다.   
1. Q, K, V는 단어 벡터를 행으로 하는 문장 행렬이다.
2. 백터의 내적(dot product)은 벡터의 유사도를 의미한다.
3. 특정 값을 분모로 사용하는 것은 값의 크기를 조절하는 스케일링을 위함이다.   

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_15_pUfIgKn.png)
문장 행렬 $Q$, 문장 행렬 $K$를 곱하면 초록색 행렬을 얻을 수 있다. 초록생 행렬은 각 단어 벡터의 유사도(내적값)가 모두 기록된 유사도 행렬을 의미한다.   
이 유사도 값을 스케일링 해주기 위해 행렬 전체를 특정 값으로 나누고, 유사도를 0과 1사이의 값으로 normalize하기 위해 softmax 함수를 사용한다. 여기에 문장 행렬 $V$와 곱하면 어텐션 값을 얻게 된다.   
![](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_16_neA52rZ.png)

정리하자면 내적(dot product)를 통해 단어 벡터간 유사도를 구한 후 특정 값을 분모로 나눠주는 방식을 통해 Q, K의 유사도를 구하여 __Scaled Dot Product Attention__ 이라고 한다. 분모에 특정 값을 나눠주지 않았다면 dot product attention이라고 말한다.

In [ ]:
# 스케일드 닷 프로덕트 어텐션 함수
def scaled_dot_product_attention(query, key, value, mask):
  # 어텐션 가중치는 Q와 K의 닷 프로덕트
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  # 가중치를 정규화
  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # 패딩에 마스크 추가
  if mask is not None:
    logits += (mask * -1e9)

  # softmax적용
  attention_weights = tf.nn.softmax(logits, axis=-1)

  # 최종 어텐션은 가중치와 V의 닷 프로덕트
  output = tf.matmul(attention_weights, value)
  return output

print("슝=3")

# 12-6. 병렬로 어텐션 수행하기

트랜스포머의 `num_heads`라는 하이퍼파라미터 변수는 병렬적으로 몇 개의 어텐션 연산을 수행할지 결정한다.   
![](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_18_nnOTx9p.png)   
트랜스포머의 초기 입력인 문장 행렬의 크기는 문장의 길이를 행으로, `d_model`은 열의 크기로 갖게 된다. 이렇게 입력된 문장 행렬을 `num_heads`의 수만큼 쪼개서 어텐션을 수행하고, 이렇게 얻은 `num_heads`의 개수만큼 어텐션 값 행렬을 하나로 concatenate한다. 위 그림은 `num_heads`가 8개인 경우인데 concatenate 하면서 열의 크기가 `d_model`이 된다.

### 멀티-헤드 어텐션

병렬로 어텐션을 수행하면 어떤 효과를 얻을 수 있을까?   
![](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_19_FwmaA3q.png)

위 그림은 `num_heads` 값이 8일 때, 병렬로 수행되는 어텐션이 서로 다른 셀프 어텐션 결과를 보여준다. 이렇게 어텐션을 병렬로 수행하는 것을 멀티 헤드 어텐션이라고 부른다.

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)

  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # Q, K, V에 각각 Dense를 적용
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # 병렬 연산을 위해 여러 개 만든다
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    # 스케일드 닷 프로덕트 어텐션 함수
    scaled_attention = scaled_dot_product_attention(query, key, value, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    # 어텐션 연산 후에 각 결과를 다시 연결(concatenate)
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    # 최종 결과에도 Dense를 한 번 더 적용
    outputs = self.dense(concat_attention)

    return outputs
print("슝=3")

# 12-7. 마스킹

특정 값들을 가려 실제 연산에 방해 되지 않도록 하는 것을 마스킹이라고 부른다. 트랜스포머에서는 어텐션을 위해 크게 두 가지 padding masking, look-ahead masking을 사용한다.   

__Padding Masking__   
패딩 토큰을 이용한 마스킹 방법이다. 자연어 처리에서 패딩은 문장의 길이가 서로 다를 때 문장의 길이를 동일하게 해주는 과정에서 짧은 문장에 숫자 0을 채워 길이를 맞춰주는 전처리 방법이다. 주어진 숫자 0은 실제 의미가 있는 단어는 아니여서 실제 어텐션과 같은 연산에서 제외할 필요가 있다. 패딩 마스킹은 숫자 0의 위치를 체크한다.

In [ ]:
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, sequence length)
  return mask[:, tf.newaxis, tf.newaxis, :]
print("슝=3")

In [ ]:
print(create_padding_mask(tf.constant([[1, 2, 0, 3, 0], [0, 0, 0, 4, 5]])))

숫자가 0인 부분을 체크한 벡터를 리턴한다. 결과를 보면 두 정수 시퀀스에 대해 결과가 출력되는데, 숫자 0인 위치에서만 1이 나오고 아닌 위치에서는 0 벡터를 출력한다. 어텐션 연산할 때 패딩 마스킹을 참고하면 숫자 0을 참고하지 않게 할 수 있다.   

__Look-ahead masking__   
순환 신경망, RNN과 트랜스포머는 문장을 입력받을 때 입력받는 방법이 전혀 다르다. RNN은 step의 개념으로 각 step마다 단어 순서대로 입력받는 구조인데 트랜스포머는 문장 행렬을 만들어 한번에 입력받는 구조이다. 그래서 이 특징 때문에 masking이 필요하다.   
자신보다 다음에 나올 단어를 참고하지 않도록 가리는 기법이 look ahead masking 기법이다. Query 단어 뒤에 나오는 Key 단어들에 마스킹 한다.   
![](https://d3s0tskafalll9.cloudfront.net/media/images/_.max-800x600.png)

행을 Query 열을 Key로 표현된 행렬이고 빨간색으로 색칠된 부분이 마스킹이다. 실제 어텐션 연산에서 가리는 역할을 하고 어텐션 연산 시에 현재 단어를 기준으로 이전 단어들과 유사도를 구할 수 있다.   

예를들어 Query 단어가 '찾고'이면 여기 있는데 행에 `<s>, <나는>, <행복을>, <찾고>`를 갖고있고 뒤에는 마스킹이 되어있다. 

In [ ]:
def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x)
  return tf.maximum(look_ahead_mask, padding_mask)
print("슝=3")

In [ ]:
print(create_look_ahead_mask(tf.constant([[1, 2, 3, 4, 5]])))

마스킹과 패딩 마스킹은 별개이므로 숫자 0인 단어가 있으면 패딩을 해야한다. 다음과 같이 적용하자.

In [ ]:
print(create_look_ahead_mask(tf.constant([[0, 5, 1, 5, 5]])))

# 12-8. 인코더

트랜스포머의 인코더 층은 셀프 어텐션과 피드포워드 신경망 총 2개의 sublayer로 구성되어 있다. 셀프 어텐션 레이어는 멀티 헤드 어텐션으로 병렬적으로 이루어져있다.   
![](https://d3s0tskafalll9.cloudfront.net/media/images/Untitled_21_Y7Cy8sm.max-800x600.png)   

두 개의 서브 층을 가지는 하나의 인코더 층을 구현하는 함수를 살펴보자. 함수 내부적으로 첫 번쨰 서브 층과 두 번째 서브 층을 구현하고 있다.

In [ ]:
# 인코더 하나의 레이어를 함수로 구현.
# 이 하나의 레이어 안에는 두 개의 서브 레이어가 존재한다
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")

  # 패딩 마스크
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })

  # 어텐션의 결과는 Dropout과 Layer Normalization이라는 훈련을 돕는 테크닉을 수행
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  # 두 번째 서브 레이어 : 2개의 완전연결층
  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 완전연결층의 결과는 Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)
print("슝=3")

![](https://d3s0tskafalll9.cloudfront.net/media/images/Untitled_22_teJgoCi.max-800x600.png)

이렇게 구현한 인코더 층을 임베딩 층과 포지서녕 인코딩을 연결하고, 사용자가 원하는 만큼 인코더 층을 쌓으면 트랜스포머의 인코더가 완성된다.   
인코더와 디코더 내부에는 서브층 이후 훈련을 돕는 layer normalization이 사용됐다. 위 그림에서 normalize에 해당하는 부분이다.   
트랜스포머는 num_layers 개수의 인코더 층을 쌓는다. 논문에서는 6개의 인코더 층을 사용했다.

In [ ]:
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  # 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 임베딩 레이어
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

  # 포지셔널 인코딩
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  # num_layers만큼 쌓아올린 인코더의 층.
  for i in range(num_layers):
    outputs = encoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)
print("슝=3")

# 12-9. 디코더

![](https://d3s0tskafalll9.cloudfront.net/media/images/Untitled_23_vBHZ3i0.max-800x600.png)

디코더의 세 개의 서브 층으로 구성되어 있다. 첫 번쨰는 셀프 어텐션, 두 번째는 인코더-디코더 어텐션, 세 번째는 피드 포워드 신경망이다. 인코더-디코더 어텐션은 Query가 디코더의 벡터인 반면에 Key와 Value가 인코더의 벡터다. 이 부분이 인코더가 입력 문장으로부터 정보를 디코더에 전달하는 과정이다.   
![](https://d3s0tskafalll9.cloudfront.net/media/images/Untitled_24_Kj9egLY.max-800x600.png)

인코더의 셀프 어텐션와 마찬가지로 디코더의 셀프 어텐션, 인코더-디코더 어텐션 두 개의 어텐션 모두 scaled dot product attention을 multi head attention으로 병렬적으로 수행한다.

In [ ]:
# 디코더 하나의 레이어를 함수로 구현.
# 이 하나의 레이어 안에는 세 개의 서브 레이어가 존재합니다.
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })

  # 멀티 헤드 어텐션의 결과는 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

  # 두 번째 서브 레이어 : 마스크드 멀티 헤드 어텐션 수행 (인코더-디코더 어텐션)
  attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1,
          'key': enc_outputs,
          'value': enc_outputs,
          'mask': padding_mask
      })

  # 마스크드 멀티 헤드 어텐션의 결과는
  # Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
  attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

  # 세 번째 서브 레이어 : 2개의 완전연결층
  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 완전연결층의 결과는 Dropout과 LayerNormalization 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)
print("슝=3")

### 디코더 층을 쌓아 디코더 만들기

위처럼 구현한 디코더 층은 임베딩 층과 포지셔널 인코딩을 연결하고, 사용자가 원하는 만큼 디코더 층을 쌓아 트랜스포머의 디코더가 완성된다. 인코더와 동일하게 `num_layers` 개수의 디코터 층을 쌓는다. 논문에서는 6개의 디코더 층을 사용했다.

In [ ]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='decoder'):
  inputs = tf.keras.Input(shape=(None,), name='inputs')
  enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')

  # 패딩 마스크
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')
  
  # 임베딩 레이어
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

  # 포지셔널 인코딩
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  # Dropout이라는 훈련을 돕는 테크닉을 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = decoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name='decoder_layer_{}'.format(i),
    )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)
print("슝=3")

# 12-10. 챗봇의 병렬 데이터 받아오기

__Cornell Movie-Dialogs Corpus__ 라는 영화 및 TV 프로그램에서 사용된 대화의 쌍을 데이터셋으로 사용한다. 기본적으로 먼저 말하는 사람의 대화 문장이 있고, 그에 응답하는 대화 문장의 쌍으로 이루어져있다.   
데이터를 받아올 때 다음과 같은 전처리를 해주자.   
- 정해진 50,000개의 질문과 답변의 쌍을 추출한다. 
- 문장에서 단어와 구두점 사이에 공백을 추가한다. 
- 알파벳과 !, ?, ,, . 총 4개의 구두점을 제외하고 다른 특수문자는 제거한다.   

데이터를 불러와보자.

In [ ]:
path_to_zip = tf.keras.utils.get_file(
    'cornell_movie_dialogs.zip',
    origin='http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip',
    extract=True)

path_to_dataset = os.path.join(
    os.path.dirname(path_to_zip), "cornell movie-dialogs corpus")

path_to_movie_lines = os.path.join(path_to_dataset, 'movie_lines.txt')
path_to_movie_conversations = os.path.join(path_to_dataset,'movie_conversations.txt')
print("슝=3")

In [ ]:
# 사용할 샘플의 최대 개수
MAX_SAMPLES = 50000
print(MAX_SAMPLES)

전처리를 위한 함수를 선언해보자. 정규 표현식을 사용하여 구두점을 제거하여 단어 토크나이징에 방해가 되지 않도록 정제하자.

In [ ]:
# 전처리 함수
def preprocess_sentence(sentence):
  sentence = sentence.lower().strip()

  # 단어와 구두점(punctuation) 사이의 거리를 만듭니다.
  # 예를 들어서 "I am a student." => "I am a student ."와 같이
  # student와 온점 사이에 거리를 만듭니다.
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
  sentence = re.sub(r'[" "]+', " ", sentence)

  # (a-z, A-Z, ".", "?", "!", ",")를 제외한 모든 문자를 공백인 ' '로 대체합니다.
  sentence = re.sub(r"[^a-zA-Z?.!,]+", " ", sentence)
  sentence = sentence.strip()
  return sentence
print("슝=3")

In [ ]:
# 질문과 답변의 쌍인 데이터셋을 구성하기 위한 데이터 로드 함수
def load_conversations():
  id2line = {}
  with open(path_to_movie_lines, errors='ignore') as file:
    lines = file.readlines()
  for line in lines:
    parts = line.replace('\n', '').split(' +++$+++ ')
    id2line[parts[0]] = parts[4]

  inputs, outputs = [], []
  with open(path_to_movie_conversations, 'r') as file:
    lines = file.readlines()

  for line in lines:
    parts = line.replace('\n', '').split(' +++$+++ ')
    conversation = [line[1:-1] for line in parts[3][1:-1].split(', ')]

    for i in range(len(conversation) - 1):
      # 전처리 함수를 질문에 해당되는 inputs와 답변에 해당되는 outputs에 적용.
      inputs.append(preprocess_sentence(id2line[conversation[i]]))
      outputs.append(preprocess_sentence(id2line[conversation[i + 1]]))

      if len(inputs) >= MAX_SAMPLES:
        return inputs, outputs
  return inputs, outputs
print("슝=3")

In [ ]:
# 데이터를 로드하고 전처리하여 질문을 questions, 답변을 answers에 저장한다.
questions, answers = load_conversations()
print('전체 샘플 수 :', len(questions))
print('전체 샘플 수 :', len(answers))

In [ ]:
print('전처리 후의 22번째 질문 샘플: {}'.format(questions[21]))
print('전처리 후의 22번째 답변 샘플: {}'.format(answers[21]))

# 12-11. 병렬 데이터 전처리하기

전처리 과정을 요약하면 다음과 같다.   
- TensorFlow Datasets SubwordTextEncoder를 토크나이저로 사용하자. 단어보다 작은 단위인 Subword를 기준으로 토크나이징하고, 각 토큰을 고유한 정수로 인코딩한다. 
- 각 문장을 토큰화하고 시작과 끝에 `START_TOKEN`, `END_TOKEN`을 추가한다. 
- 최대 길이 MAX_LENGTH 40을 넘으면 필터링한다.
- MAX_LENGTH보다 길이가 짧으면 40에 맞도록 패딩한다.

### 1. 단어장 만들기   
각 단어에 고유한 정수 인덱스를 부여하기 위해 단어장을 만들자. 질문과 답변 데이터셋 모두 사용하자. 시작 토큰과 종료 토큰도 단어장에 추가해서 정수를 부여하자. 각각 단어장의 크기보다 +1된 큰 수를 부여해 적용하자.

In [ ]:
import tensorflow_datasets as tfds
print("살짝 오래 걸릴 수 있어요. 스트레칭 한 번 해볼까요? 👐")

# 질문과 답변 데이터셋에 대해서 Vocabulary 생성. (Tensorflow 2.3.0 이상) (클라우드는 2.4 입니다)
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(questions + answers, target_vocab_size=2**13)
print("슝=3 ")

In [ ]:
import tensorflow_datasets as tfds
print("슝=3 ")

In [ ]:
# 시작 토큰과 종료 토큰에 고유한 정수를 부여한다.
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]
print("슝=3")

In [ ]:
print('START_TOKEN의 번호 :' ,[tokenizer.vocab_size])
print('END_TOKEN의 번호 :' ,[tokenizer.vocab_size + 1])

In [ ]:
# 시작 토큰과 종료 토큰을 고려하여 +2를 하여 단어장의 크기를 산정합니다.
VOCAB_SIZE = tokenizer.vocab_size + 2
print(VOCAB_SIZE)

### 2. 각 단어를 고유한 정수로 인코딩, 패딩하기   
`tokenizer.encode()`로 단어를 정수로 변환할 수 있고 `tokenizer.decode()`로 정수 시퀀스를 단어 시퀀스로 변환할 수 있다. MAX_LENGTH가 40이 넘는 애들은 필터링해주자.

In [ ]:
# 임의의 22번째 샘플에 대해서 정수 인코딩 작업을 수행.
# 각 토큰을 고유한 정수로 변환
print('정수 인코딩 후의 21번째 질문 샘플: {}'.format(tokenizer.encode(questions[21])))
print('정수 인코딩 후의 21번째 답변 샘플: {}'.format(tokenizer.encode(answers[21])))

In [ ]:
# 샘플의 최대 허용 길이 또는 패딩 후의 최종 길이
MAX_LENGTH = 40
print(MAX_LENGTH)

In [ ]:
# 정수 인코딩, 최대 길이를 초과하는 샘플 제거, 패딩
def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []
  
  for (sentence1, sentence2) in zip(inputs, outputs):
    # 정수 인코딩 과정에서 시작 토큰과 종료 토큰을 추가
    sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

    # 최대 길이 40 이하인 경우에만 데이터셋으로 허용
    if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:
      tokenized_inputs.append(sentence1)
      tokenized_outputs.append(sentence2)
  
  # 최대 길이 40으로 모든 데이터셋을 패딩
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')
  
  return tokenized_inputs, tokenized_outputs
print("슝=3")

In [ ]:
questions, answers = tokenize_and_filter(questions, answers)
print('단어장의 크기 :',(VOCAB_SIZE))
print('필터링 후의 질문 샘플 개수: {}'.format(len(questions)))
print('필터링 후의 답변 샘플 개수: {}'.format(len(answers)))

### 3. 교사 강요(Teacher Forcing) 사용하기   
tf.data.Dataset API는 훈련 프로세스의 속도가 빨라지도록 입력 파이프라인을 구축하는 API이다. 이를 사용하기 위해 질문과 답변의 쌍을 `tf.data.Dataset`의 입력으로 넣어주는 작업을 한다.   
디코더의 입력과 실제값(레이블)을 정의해주기 위해 교사 강요(Teacher Forcing)이라는 언어 모델의 훈련 기법을 이해해야한다. [RNN 언어 모델](https://wikidocs.net/46496)을 읽고 한번 교사 강요에 대해 알아보자.   
> 테스트 과정에서 t 시점의 출력이 t+1 시점의 입력으로 사용되는 RNN 모델을 훈련시킬 때 사용하는 훈련 기법입니다. Teacher Forcing을 사용하지 않은 경우 잘못된 예측이 다음 시점의 입력으로 들어가면서 연쇄적으로 예측에 정확도를 미친다.   

자신의 상태를 결정하는 모델을 자기회귀 모델(auto-regressive model, AR)이라고 한다. RNN 언어 모델은 대표적인 자기 회귀 모델의 예시이며, 트랜스포머의 디코더도 자기 회귀 모델이다. 트랜스포머의 디코더에 교사 강요(teacher forcing)를 적용한다.   

질문과 답변의 쌍을 tf.data.Dataset API의 입력으로 사용하여 파리프라인을 구성하자. 교사 강요를 위해 `answers[:, :-1]`를 디코더의 입력값, `answers[:, 1:]`를 디코더의 레이블로 사용하자.

In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

# 디코더는 이전의 target을 다음의 input으로 사용합니다.
# 이에 따라 outputs에서는 START_TOKEN을 제거하겠습니다.
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,
        'dec_inputs': answers[:, :-1]
    },
    {
        'outputs': answers[:, 1:]
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
print("슝=3")